In [227]:
from utils import simulate_fair_price, track_pnl
import utils as utl
from order_book import OrderBook
from trade import Trade

import plotly.express as px
import polars as pl
import numpy as np

## Question 1 : simulating fair prices

In [228]:
historical_fair_price = simulate_fair_price()
historical_fair_price

timestamp,Fair Price
datetime[μs],f64
2025-01-01 00:00:00,100.0
2025-01-01 01:00:00,99.993465
2025-01-01 02:00:00,99.924112
2025-01-01 03:00:00,99.956111
2025-01-01 04:00:00,100.081577
…,…
2025-12-31 20:00:00,15.641663
2025-12-31 21:00:00,15.61366
2025-12-31 22:00:00,15.610055


In [229]:
historical_bid_ask = utl.compute_all_bid_ask(
    historical_fair_price,
    500_000_000
)
historical_bid_ask

timestamp,fair_price,log_return,volatility,spread,bid,ask
datetime[μs],f64,f64,f64,f64,f64,f64
2025-01-01 00:00:00,100.0,null,null,null,null,null
2025-01-01 01:00:00,99.993465,-0.000065,0.0,0.0,99.993465,99.993465
2025-01-01 02:00:00,99.924112,-0.000694,0.000314,0.000628,99.908086,99.940138
2025-01-01 03:00:00,99.956111,0.00032,0.000418,0.000836,99.934798,99.977425
2025-01-01 04:00:00,100.081577,0.001254,0.000706,0.001413,100.045555,100.117599
…,…,…,…,…,…,…
2025-12-31 20:00:00,15.641663,0.000256,0.088373,0.176745,11.134658,20.148669
2025-12-31 21:00:00,15.61366,-0.001792,0.088368,0.176735,11.106912,20.120408
2025-12-31 22:00:00,15.610055,-0.000231,0.088363,0.176725,11.103564,20.116546


In [230]:
utl.generate_market_order(historical_bid_ask)


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


timestamp,fair_price,log_return,volatility,spread,bid,ask,prob_trade_bid,prob_trade_ask
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64
2025-01-01 00:00:00,100.0,null,null,null,null,null,null,null
2025-01-01 01:00:00,99.993465,-0.000065,0.0,0.0,99.993465,99.993465,0.632121,0.632121
2025-01-01 02:00:00,99.924112,-0.000694,0.000314,0.000628,99.908086,99.940138,0.611497,0.611497
2025-01-01 03:00:00,99.956111,0.00032,0.000418,0.000836,99.934798,99.977425,0.604703,0.604703
2025-01-01 04:00:00,100.081577,0.001254,0.000706,0.001413,100.045555,100.117599,0.585858,0.585858
…,…,…,…,…,…,…,…,…
2025-12-31 20:00:00,15.641663,0.000256,0.088373,0.176745,11.134658,20.148669,1.4100e-7,1.4100e-7
2025-12-31 21:00:00,15.61366,-0.001792,0.088368,0.176735,11.106912,20.120408,1.4113e-7,1.4113e-7
2025-12-31 22:00:00,15.610055,-0.000231,0.088363,0.176725,11.103564,20.116546,1.4125e-7,1.4125e-7


Question for teacher : what is the economic reality behind these models / what are some good papers explaining these choices
- for a currency, going to 0.01 for a few months and then 50 months later is not realistic / signifies huge macroeconomic events
- what are some papers indicating which parameters make sense in this context

## Question 2 : Order book

In [231]:
# Example Usage
order_book = OrderBook(n_levels=5)
order_book.update_order(price=100.5, size=10, side="bid")
order_book.update_order(price=101.0, size=15, side="bid")
order_book.update_order(price=102.0, size=5, side="ask")
order_book.update_order(price=102.5, size=20, side="ask")
order_book.update_order(price=103.0, size=8, side="ask")

print("Best Bid:", order_book.get_best_bid())  # (101.0, 15)
print("Best Ask:", order_book.get_best_ask())  # (102.0, 5)
print("Order Book:\n", order_book.get_order_book())

Best Bid: (101.0, 15.0, datetime.time(19, 28, 27, 587082), False)
Best Ask: (102.0, 5.0, datetime.time(19, 28, 27, 587955), False)
Order Book:
 shape: (3, 8)
┌────────────┬─────────────────┬──────────┬───────┬───────┬──────────┬────────────────┬────────────┐
│ client_bid ┆ timestamp_bid   ┆ size_bid ┆ bid   ┆ ask   ┆ size_ask ┆ timestamp_ask  ┆ client_ask │
│ ---        ┆ ---             ┆ ---      ┆ ---   ┆ ---   ┆ ---      ┆ ---            ┆ ---        │
│ bool       ┆ time            ┆ f64      ┆ f64   ┆ f64   ┆ f64      ┆ time           ┆ bool       │
╞════════════╪═════════════════╪══════════╪═══════╪═══════╪══════════╪════════════════╪════════════╡
│ false      ┆ 19:28:27.587082 ┆ 15.0     ┆ 101.0 ┆ 102.0 ┆ 5.0      ┆ 19:28:27.58795 ┆ false      │
│            ┆                 ┆          ┆       ┆       ┆          ┆ 5              ┆            │
│ false      ┆ 19:28:27.586128 ┆ 10.0     ┆ 100.5 ┆ 102.5 ┆ 20.0     ┆ 19:28:27.58835 ┆ false      │
│            ┆                 ┆  

## Question 4 : Trade logic

In [232]:
trade = Trade(26, "buy")

In [233]:
order_book.get_order_book()

client_bid,timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask,client_ask
bool,time,f64,f64,f64,f64,time,bool
false,19:28:27.587082,15.0,101.0,102.0,5.0,19:28:27.587955,false
false,19:28:27.586128,10.0,100.5,102.5,20.0,19:28:27.588350,false
null,null,null,null,103.0,8.0,19:28:27.589049,false


In [234]:
ob = trade.update_orderbook_with_trade(orderbook=order_book)

In [235]:
ob.get_order_book()

client_bid,timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask,client_ask
bool,time,f64,f64,f64,f64,time,bool
false,19:28:27.587082,15.0,101.0,103.0,7.0,19:28:27.648679,false
false,19:28:27.586128,10.0,100.5,null,null,null,null


In [236]:
pl.DataFrame(trade.history)

AttributeError: 'Trade' object has no attribute 'history'

In [ ]:
track_pnl(simulate_fair_price(), pl.DataFrame(trade.history), 10_000_000)

timestamp,pnl,inventory
datetime[μs],f64,f64
2025-04-15 21:48:52.707322,0.0,1e7
2025-04-15 21:48:52.707322,74.508623,9.999999e6
2025-04-15 21:48:52.707324,148.517247,9.999998e6
2025-04-15 21:48:52.707325,222.02587,9.999997e6
2025-04-15 21:48:52.707325,222.02587,9.999997e6
